# 1.Google GenAI Provider

In [25]:
# providers/google_genai_provider.py
from google import genai

class GoogleGenAIProvider:
    def __init__(self, model="gemini-1.5-pro", api_key=None):
        self.client = genai.Client(api_key=api_key)
        self.model = model

    def extract(self, prompt: str) -> str:
        response = self.client.models.generate_content(
            model=self.model,
            contents=prompt
        )
        return response.text


# 2.Local LLM Provider

In [2]:
# providers/local_llm_provider.py
import subprocess

class LocalLLMProvider:
    def __init__(self, model="llama3"):
        self.model = model

    def extract(self, prompt: str) -> str:
        result = subprocess.run(
            ["ollama", "run", self.model, prompt],
            capture_output=True,
            text=True
        )
        return result.stdout.strip()


# 3.Docling Adapter

In [4]:
%pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 33.3 MB/s eta 0:00:00
   ━

In [5]:
# utils/docling_adapter.py
from docling.document_converter import DocumentConverter

class DoclingAdapter:
    def __init__(self):
        self.converter = DocumentConverter()

    def convert(self, file_path: str) -> str:
        doc = self.converter.convert(file_path)
        return doc.document.export_to_markdown()


# 4.Integration Pipeline

In [17]:
from google.colab import files

uploaded = files.upload()   # this will open a file picker


Saving report.pdf to report.pdf


In [28]:
file_path = "report.pdf"


In [36]:
# 1️⃣ Install required packages
!pip install docling transformers sentencepiece spacy --quiet
!python -m spacy download en_core_web_sm --quiet

# 2️⃣ Imports
from docling.document_converter import DocumentConverter
from transformers import pipeline
import spacy

# 3️⃣ PDF → text conversion
file_path = "report.pdf"  # make sure the file is uploaded

converter = DocumentConverter()
doc = converter.convert(file_path)
text = doc.document.export_to_markdown()

print("✅ Text extracted from PDF (first 500 chars):\n")
print(text[:500], "...")

# 4️⃣ Setup summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# 5️⃣ Chunking function
def chunk_text(text, chunk_size=1000):
    """Split text into chunks of at most chunk_size characters."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# 6️⃣ Summarize chunks
chunks = chunk_text(text, chunk_size=1000)
summaries = []

for i, chunk in enumerate(chunks):
    print(f"⏳ Summarizing chunk {i+1}/{len(chunks)}...")
    s = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
    summaries.append(s[0]['summary_text'])

final_summary = " ".join(summaries)
print("\n✅ Final Summary:\n")
print(final_summary)

# 7️⃣ NER / entity extraction using spaCy
nlp = spacy.load("en_core_web_sm")
doc_nlp = nlp(text)

entities = []
for ent in doc_nlp.ents:
    entities.append({"text": ent.text, "label": ent.label_})

print("\n✅ Extracted Entities:\n")
for e in entities:
    print(e)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ Text extracted from PDF (first 500 chars):

## Press Release

2 nd April, 2025 Accra

## Government of Ghana convenes National Landscape Forum on Natural Resource Management and Forest Economy for inclusive and sustainable growth

The Government of Ghana (GoG), with support from the World Bank, has organized a 3-day National Landscape Forum, a programme which aimed to bring together professionals and interested persons from  public,  private  and  non-governmental  agencies  working  in  the  areas  of  natural  resources management (NRM) ...


Device set to use cpu


⏳ Summarizing chunk 1/8...
⏳ Summarizing chunk 2/8...


Your max_length is set to 200, but your input_length is only 191. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=95)


⏳ Summarizing chunk 3/8...


...

⏳ Summarizing chunk 5/8...
⏳ Summarizing chunk 6/8...


Your max_length is set to 200, but your input_length is only 176. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=88)


⏳ Summarizing chunk 7/8...


Your max_length is set to 200, but your input_length is only 156. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=78)


⏳ Summarizing chunk 8/8...

✅ Final Summary:

The Government of Ghana (GoG), with support from the World Bank, has organized a 3-day National Landscape Forum. The forum aimed to bring together professionals and interested persons from  public,  private  and  non-governmental  agencies. Main environmental challenges in Ghana include deforestation, land and landscape degradation, pollution, coastal degradation, and depletion of fish. The GoG has rolled out several programs to support the sustainable management of its natural resources and environmental restoration. The World Bank has supported the Government in developing the land accounts, the ecosystem extent accounts, and the ecosystem services accounts. These accounts present changes in land and ecosystem cover and provide physical and monetary value estimates. Ghana is one of the first countries globally to receive carbon payments from the World Bank for emissions reductions generated in the cocoa forest mosaic landscape. Ghana's in